# ↓23/8/17

In [6]:
# 読込み用の関数
def read_bin(filename):
    with open(filename,'rb') as f:
        return pickle.load(f)
    
def save_bin(data,filename):
    with open(filename,'wb') as f:
        pickle.dump(data,f)

### 構文木情報， tokenとtagのjson情報を読込

In [135]:
import pickle

train_trees = []
for num in range(0, 131501, 500):
    with open('data/Stanford_coreNLP/train_'+str(num)+'-'+str(num+499)+'.json_tree.binaryfile', 'rb') as f:
        a_train_tree = pickle.load(f)
        train_trees.extend(a_train_tree)

In [160]:
import json

train = []
with open("data/f_id/train.json","r") as f:
    for line in f:
        train.append(json.loads(line))
    
# id検索用の辞書作成
idkey_train = {}
for d in train:
    idkey_train[d["id"]] = d
    
del train

### 構文解析結果の処理に必要な関数

In [ ]:
import copy
from nltk.tree import Tree

#部分木から具体的な単語を削除
def remove_words(node):
    for i, child in enumerate(node):
        if isinstance(child, str):  # 葉ノード（単語）の場合
            node[i] = ''  # 単語を空文字に置換
        elif isinstance(child, Tree):
            remove_words(child)

# 各部分木、Treeタグ、トークンリスト、深さを取得
def get_all_subtree_info(node, depth=0):
    
#     subtree用
#     node_cp = copy.deepcopy(node)
#     remove_words(node_cp)
    subtree_tag = node.label()
    subtree_tokens = [child for child in node.leaves()]
    subtree_depth = depth

    # 部分木の情報を辞書として格納
    subtree_info = {
#         'subtree': node_cp,
        'tree_tag': subtree_tag,
        'tokens': subtree_tokens,
        'depth': subtree_depth
    }
    # 子ノードに対して再帰的に情報を収集
    child_subtree_info = []
    for child in node:
        if isinstance(child, Tree):
            child_subtree_info.extend(get_all_subtree_info(child, depth + 1))
            
    # 子ノードの情報を追加
    subtree_info['children'] = child_subtree_info
    return [subtree_info]


# 全ての部分木情報をリスト内の対等な深さに格納
def display_subtree_info(subtree_info, all_subtree):
    new_d = copy.deepcopy(subtree_info)
    
    if "children" in new_d:
        del new_d["children"]
     
    all_subtree.append(new_d)
    for child_info in subtree_info.get('children', []):
        display_subtree_info(child_info,all_subtree)

### 部分木リストall_subtree（ner-tagを付ける前）を取得

In [ ]:
import pickle

all_subtree = []
n = len(train_trees)
for i,tree in enumerate(train_trees):
    print('\r%d / %d' %(i, n), end='')
    
    # 全ての部分木情報を取得
    all_subtree_info = get_all_subtree_info(tree)
    subtree = []
    
    # 全ての部分木をリスト内の対等な深さに格納
    for subtree_info in all_subtree_info:
        display_subtree_info(subtree_info,subtree)
    for d in subtree:
        d["id"] = i
    all_subtree.extend(subtree)
# subtreeを含む場合の具体的単語削除    
# for d in all_subtree:
#     remove_words(d["subtree"])

# 書き出し
# with open("temp/all_subtree.binaryfile", 'wb') as f:
#     pickle.dump(all_subtree,f)

# 読込み
# with open("temp/all_subtree_rm_at.binaryfile",'rb') as f:
#     all_subtree = pickle.load(f)

In [44]:
# all_subtreeのうち，交換対象にふさわしくないtree_tagを持つ部分木の削除
　# 木全体であるROOT，'．''，'-LRB-','-RRB-'等tree_tag = tokenで種類がない物
with open("temp/all_subtree.binaryfile",'rb') as f:
    all_subtree = pickle.load(f)

remove_tag ={"''",',','-LRB-','-RRB-','.',':','``',"ROOT"}
all_subtree_rm = []
n = len(all_subtree)
for i,d in enumerate(all_subtree):
    print('\r%d / %d' %(i, n), end='')
    if d["tree_tag"] not in remove_tag:
        all_subtree_rm.append(d)
        
# with open("temp/all_subtree_rm.binaryfile", 'wb') as f:
#     pickle.dump(all_subtree_rm,f)

5849551 / 5849552

In [47]:
# all_subtree_rmのうち試験的に単一トークンの削除 (remove a token)
all_subtree_rm_at = []

n = len(all_subtree_rm)
for i,d in enumerate(all_subtree_rm):
    print('\r%d / %d' %(i, n), end='')
    if len(d["tokens"]) != 1:
        all_subtree_rm_at.append(d)

#all_subtree_rm_atを格納
# with open("temp/all_subtree_rm_at.binaryfile", 'wb') as f:
#     pickle.dump(all_subtree_rm_at,f)

5310312 / 5310313

In [230]:
# 読込み
# with open("temp/all_subtree_rm_at.binaryfile",'rb') as f:
#     all_subtree_rm_at = pickle.load(f)

In [231]:
# all_subtree_rm_atのtokenである-LRB-と-RRB-を（）に戻す
for d in all_subtree_rm_at:
    new_tokens = []
    for t in d["tokens"]:
        if t == "-LRB-":
            new_tokens.append("(")
        elif t == "-RRB-":
            new_tokens.append(")")
        else:
            new_tokens.append(t)
    d["tokens"] = new_tokens
                

#all_subtree_rm_at(exchang完了済)を格納
# with open("temp/all_subtree_rm_at_exc.binaryfile", 'wb') as f:
#     pickle.dump(all_subtree_rm_at,f) 

In [232]:
# all_subtree = all_subtree_rm_at
# del all_subtree_rm
# del all_subtree_rm_at

### 部分木リストall_subtreeに対応するner-tag情報を付与

In [221]:
# 指定部分tokenに対応するtag取得関数

def get_tags_for_tokens(sub_tokens,d):
    match_cnt = 0 #トークン列の連続が複数含まれていた場合の対処
    sub_tags_and_index = dict()
    n = len(d["tokens"])
    n_sub = len(sub_tokens)
    
    # d["token"]列からsub_tokensの数の連続するリストを切り出しsub_tokensと比較
    for i in range(0,n-n_sub+1):#同じトークン列の繰返し出現に備え,forは止めない
        partof_d_tokens = d["tokens"][i:i+n_sub]
        if partof_d_tokens == sub_tokens:
            sub_tags_and_index["sub_tags"] = d["tags"][i:i+n_sub]
            sub_tags_and_index["start_index"] = i
            match_cnt += 1
            
    if match_cnt == 1:
        return sub_tags_and_index
    else:
        return dict()

In [196]:
# import pickle
# all_subtree=read_bin("temp/all_subtree_rm_at_exc.binaryfile")

In [233]:
# 部分木のtag情報付与 更新版
# need:idkey_train
success_subtree = []#1854621
fail_subtree = []#228083

n = len(all_subtree)
for i,sub_d in enumerate(all_subtree):#2082704
    print('\r%d / %d' %(i, n), end='')
    try:
        sub_tags_and_index = get_tags_for_tokens(sub_d["tokens"],idkey_train[sub_d["id"]])
        
        sub_d["tags"] = sub_tags_and_index["sub_tags"]
        sub_d["start_index"] = sub_tags_and_index["start_index"]
        success_subtree.append(sub_d)
        
    except Exception as e:
        fail_subtree.append(sub_d)

2082703 / 2082704

In [258]:
# save_bin(success_subtree,"temp/success_subtree.binaryfile") 
# save_bin(fail_subtree,"temp/fail_subtree.binaryfile")

### A - B をA-Bに戻す

In [322]:
success_subtree = read_bin("temp/success_subtree.binaryfile")
fail_subtree = read_bin("temp/fail_subtree.binaryfile")

In [323]:
# 指定部分tokenに対応するtag取得関数 A - BとA-Bの違い対応版

def get_tags_for_diff_len_tokens(sub_tokens,d):
    sub_tags_and_index = dict()
    sub_s = "".join(sub_tokens)
    stop_i_roop = 0
    
    for i in range(len(d["tokens"])):
        n = len(d["tokens"][i])
        if stop_i_roop == 1:
            break
            
        if sub_s[0:n] == d["tokens"][i]:
            part_of_d = d["tokens"][i]
            end = 0
            
            for j in range(i+1,len(d["tokens"])):
                if d["tokens"][j] in sub_s:
                    part_of_d += d["tokens"][j]
                    if sub_s == part_of_d:
                        end = j+1
                        stop_i_roop = 1
                        break
    
    sub_tags_and_index["sub_tokens"] = d["tokens"][i:j+1]
    sub_tags_and_index["sub_tags"] = d["tags"][i:j+1]
    sub_tags_and_index["start_index"] = i
    return sub_tags_and_index

In [324]:
fail_subtree_2 = []#5159

n = len(fail_subtree)
for i,sub_d in enumerate(fail_subtree):#228083
    print('\r%d / %d' %(i, n), end='')
    try:
        sub_tokens_tags_index = get_tags_for_diff_len_tokens(sub_d["tokens"],idkey_train[sub_d["id"]])
        sub_d["tokens"] = sub_tokens_tags_index["sub_tokens"]
        sub_d["tags"] = sub_tokens_tags_index["sub_tags"]
        sub_d["start_index"] = sub_tokens_tags_index["start_index"]
        success_subtree.append(sub_d)
        
    except Exception as e:
        fail_subtree_2.append(sub_d)

228082 / 228083

In [339]:
# success_subtreeのうち試験的に単一トークンの削除 (remove a token)
success_subtree_at = []

n = len(success_subtree)
for i,d in enumerate(success_subtree):
    print('\r%d / %d' %(i, n), end='')
    if len(d["tokens"]) != 1:
        success_subtree_at.append(d)

2077544 / 2077545

In [340]:
# save_bin(success_subtree_at,"temp/success_subtree_A-B.binaryfile")
# save_bin(fail_subtree_2,"temp/fail_subtree_A-B.binaryfile")

# ↓23/8/18
【今後の予定】  
・もとのsmallにはidがついていない＝＞3/167（ID付き）を新たに10作成　【18日完了】  
・その文番号のみのsmall_subtreeを新たに作成 【18日完了】  
・交換先の選定　【18日完了】  
・交換時の比率を決定＝＞1文につき1か所  【18日完了】  
・交換候補からROOTや.を削除 【19日完了】  
・交換後の文生成    

### id付きの3/167データ×10を新たに準備

In [ ]:
# train = []
# with open("data/f_id/train.json","r") as f:
#     for line in f:
#         train.append(json.loads(line))

In [ ]:
labels = ["art-broadcastprogram","art-film","art-music","art-other","art-painting","art-writtenart","building-airport","building-hospital","building-hotel","building-library","building-other","building-restaurant","building-sportsfacility","building-theater","event-attack/battle/war/militaryconflict","event-disaster","event-election","event-other","event-protest","event-sportsevent","location-GPE","location-bodiesofwater","location-island","location-mountain","location-other","location-park","location-road/railway/highway/transit","organization-company","organization-education","organization-government/governmentagency","organization-media/newspaper","organization-other","organization-politicalparty","organization-religion","organization-showorganization","organization-sportsleague","organization-sportsteam","other-astronomything","other-award","other-biologything","other-chemicalthing","other-currency","other-disease","other-educationaldegree","other-god","other-language","other-law","other-livingthing","other-medical","person-actor","person-artist/author","person-athlete","person-director","person-other","person-politician","person-scholar","person-soldier","product-airplane","product-car","product-food","product-game","product-other","product-ship","product-software","product-train","product-weapon","O"]
# 66種のラベルごとに対応するリスト ラベルを含むtrainの各要素を含むリストが入っている値
labeled_data = []
for label in labels:
    new_l = []
    for d in train:
        if label == "O":
            if {"O"} == set(d["tags"]):
                new_l.append(d)
        else:
            if label in set(d["tags"]):
                new_l.append(d)
    labeled_data.append(new_l)#listに保存

In [ ]:
#3/167スケールのデータを新たに10作成（id付き）
import random
n = 3
for ver in range(0,10):
    labeled_data_elected = []#21548
    for i in range(len(labeled_data)):
        choice_num = int(len(labeled_data[i])*n/166)
        print(choice_num,"/",len(labeled_data[i]),end = "    ")
        choice_list = random.sample(labeled_data[i],choice_num)
        labeled_data_elected.extend(choice_list)
    print(len(labeled_data_elected))

    #f_smallを作成
    with open("data/f_small_id_3-167/original/"+str(ver)+".json", 'w') as f:
        for d in labeled_data_elected:
            f.write(json.dumps(d))
            f.write("\n")

# ↓23/8/19
【今後の予定】  
・きちんと部分一致するtoken群を探すべき【20日完了】
→ tokenひとつひとつを検索した場合，一番最初の全く違うトークンを拾ってきてしまう可能性がある  
・交換後の文生成【20日完了】

### 新たに生成したデータの読み込み，DA加工

In [1]:
### 深さとタグから候補部分木をリストで返す関数

# 引数であるsentidにset又はidをとり，sentidに一致するidのsubreeだけを返す関数
def search_d_sentid(sbts_list,q_id):# max(depth):34
    found_sbts = []
    for d in sbts_list:
        if type(q_id) == set:
            if d["id"] in q_id:
                found_sbts.append(d)
        elif type(q_id) == int:
            if d["id"] == q_id:
                found_sbts.append(d)
    return found_sbts

# 深さとタグから候補部分木をリストで返す関数
def search_d_depth_and_tag(sbts_list,q_depth,q_tree_tag):# max(depth):34
    found_sbts = []
    for d in sbts_list:
        if d["depth"] == q_depth and d["tree_tag"] == q_tree_tag:
            found_sbts.append(d)
    return found_sbts

# タグのみから候補部分木をリストで返す関数
def search_d_tag(sbts_list,q_tree_tag):# max(depth):34
    found_sbts = []
    for d in sbts_list:
        if d["tree_tag"] == q_tree_tag:
            found_sbts.append(d)
    return found_sbts

In [2]:
# 指定部分木tokenを交換用部分木tokenに入れ替え，tagを対応させた辞書を返す関数
def swap_tokens(f_d,rand_sbt_in_s,rand_exch_pair):
    new_d = dict()
    i = rand_sbt_in_s["start_index"]
    n = len(rand_sbt_in_s["tokens"])
    new_d["tokens"] = f_d["tokens"][:i]+rand_exch_pair["tokens"]+f_d["tokens"][i+n:]
    new_d["tags"] = f_d["tags"][:i]+rand_exch_pair["tags"]+f_d["tags"][i+n:]
#     new_d["ids"] = f_d["id"]
    return new_d

In [8]:
import pickle
subtrees = read_bin("temp/success_subtree_A-B.binaryfile")

In [14]:
import json
import random

no_sbt_s = set()
no_exc_sbt = set()

for n in range(1,10):
    train_f = []# 該当ファイルのjsonデータ
    with open("data/f_small_id_3-167/original/"+str(n)+".json", "r") as f:
        for line in f:
            train_f.append(json.loads(line))        
    len_train = len(train_f)
    ids_in_f = set([d["id"] for d in train_f])
    sbts_in_f = [d for d in subtrees if d["id"] in ids_in_f]
    
    data_f_path = "data/SubTree/"+str(n)+"_x10.json"
    log_f_path = "log/SubTree_DA_log/"+str(n)+"_log.txt"
    
    with open(data_f_path,"w",encoding='utf-8') as data_f, open(log_f_path,"w",encoding='utf-8') as log_f:
        for i,d in enumerate(train_f):
            print('\r(%d) %d / %d' %(n,i,len_train), end='')

            data_f.write(json.dumps({"tokens":d["tokens"],"tags":d["tags"]}))#データを含める
            data_f.write("\n")        
            log_f.write("★ "+" ".join(d["tokens"])+"\n")# 元データのログ

            for _ in range(10):
                sbts_in_s = search_d_sentid(sbts_in_f,d["id"])# 部分木群
                
                if len(sbts_in_s) == 0:# 部分木の無い文を報告
                    no_sbt_s.add(d["id"])
                    log_f.write(" 　【no-subtree-ERROR】[id:"+str(d["id"])+"]\n")# 交換のログ出力
                    break
                else:
                    rand_sbt = random.choice(sbts_in_s)# ランダム部分木一つ
                    
                    exch_pairs = search_d_depth_and_tag(
                        sbts_in_f,rand_sbt["depth"],rand_sbt["tree_tag"])# 交換候補群
                    
                    if len(exch_pairs) == 0:# 交換候補の無い部分木を報告
                        no_exc_sbt.add(rand_sbt)
                        log_f.write(" 　【no-pair-ERROR】["+" ".join(rand_sbt["tokens"])+"]\n")# 交換のログ出力
                        break
                    else:
                        rand_exch_pair = random.choice(exch_pairs)# ランダム交換候補一つ
                        
                        log_f.write(" 　・ "+" ".join(rand_sbt["tokens"])+
                        " => "+" ".join(rand_exch_pair["tokens"])+"\n")# 交換のログ出力

                        new_d = swap_tokens(d,rand_sbt,rand_exch_pair)
                        data_f.write(json.dumps(new_d))
                        data_f.write("\n")

(9) 4191 / 4192

# 23/8/20 & 21　

【今後の予定】  
・エラーで止まってしまったので対応【20日完了】  
・エラーのチェック → A-Bのエラー修正【21日完了】 
・データ完成 【21日完了】  
・データをFlowに送り学習【22日完了】  